Gather all tickers

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from typing import List, Tuple

wiki_page: str = requests.get('https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds').text
soup: BeautifulSoup = BeautifulSoup(wiki_page,'lxml')

list_items = soup.select('li:contains("|")')
tickers: List[str] = []

for list_item in list_items:
    li_text: str = list_item.text
    start_index: int = li_text.find('|')
    end_index: int = li_text.find(')',start_index)
    tickers.append(li_text[start_index +1:end_index].strip())

tickers.append('^GSPTSE') # add S&P/TSX Composite as benchmark
print(tickers)

['ITOT', 'ACWI', 'IWV', 'SCHB', 'FNDB', 'VT', 'VTI', 'VXUS', 'VTHR', 'DIA', 'RSP', 'IOO', 'IVV', 'SPY', 'SHE', 'VOO', 'IWM', 'OEF', 'QQQ', 'CVY', 'RPG', 'RPV', 'IWB', 'IWF', 'IWD', 'IVV', 'IVW', 'IVE', 'PKW', 'PRF', 'SPLV', 'SCHX', 'SCHG', 'SCHV', 'SCHD', 'FNDX', 'SDY', 'VOO', 'VOOG', 'VOOV', 'VV', 'VUG', 'VTV', 'MGC', 'MGK', 'MGV', 'VONE', 'VONG', 'VONV', 'VIG', 'VYM', 'DTN', 'DLN', 'MDY', 'DVY', 'IWR', 'IWP', 'IWS', 'IJH', 'IJK', 'IJJ', 'PDP', 'SCHM', 'IVOO', 'IVOG', 'IVOV', 'VO', 'VOT', 'VOE', 'VXF', 'DON', 'IWC', 'IWM', 'IWO', 'IWN', 'IJR', 'IJT', 'IJS', 'SCHA', 'FNDA', 'VIOO', 'VIOG', 'VIOV', 'VB', 'VBK', 'VBR', 'VTWO', 'VTWG', 'VTWV', 'EEB', 'ECON', 'IDV', 'ACWX', 'BKF', 'EFA', 'EFG', 'EFV', 'SCZ', 'EEM', 'PID', 'SCHC', 'SCHE', 'SCHF', 'FNDF', 'FNDC', 'FNDE', 'DWX', 'VEA', 'VWO', 'VXUS', 'VEU', 'VSS', 'DEM', 'DGS', 'AAXJ', 'EZU', 'EPP', 'IEV', 'ILF', 'FEZ', 'VGK', 'VPL', 'HEDJ', 'DFE', 'AND', 'GXF', 'EWA', 'EWC', 'EWG', 'EIS', 'EWI', 'EWJ', 'EWD', 'EWL', 'EWP', 'EWU', 'DXJ', 'NOR

Fetch ticker info from Yahoo

In [3]:
import yfinance as yf

data: pd.DataFrame = yf.download(tickers=" ".join(tickers), period="5y", interval="1d", group_by='ticker')
print(data)

[*********************100%***********************]  426 of 426 completed

16 Failed downloads:
- GLDE: No data found for this date range, symbol may be delisted
- QEH: No data found, symbol may be delisted
- RRF: No data found, symbol may be delisted
- IRV: No data found for this date range, symbol may be delisted
- BABZ: No data found for this date range, symbol may be delisted
- FTGS: No data found, symbol may be delisted
- ONEF: No data found for this date range, symbol may be delisted
- HDGI: No data found for this date range, symbol may be delisted
- CRDT: No data found for this date range, symbol may be delisted
- RWG: No data found, symbol may be delisted
- ACCU: No data found for this date range, symbol may be delisted
- YPRO: No data found, symbol may be delisted
- GGBP: No data found for this date range, symbol may be delisted
- RPX: No data found, symbol may be delisted
- WDTI: No data found, symbol may be delisted
- BGU: No data found for this date range, symbol may be deli

Save output to file to prevent further network requests.

In [4]:
import os

found_tickers: List[str] = data.columns.get_level_values(0).unique().to_list()

for found_ticker in found_tickers:
    data[found_ticker].to_csv(os.path.join("data", found_ticker + '.csv'))


Read files back from directory.

In [7]:
import glob

csv_paths: List[str] = glob.glob(os.path.join("data", '*.csv'))
prices_df: pd.DataFrame = None

for csv_path in csv_paths:
    (ticker_id, extension) = csv_path.split(".", 1)
    df: pd.DataFrame = pd.read_csv(csv_path, index_col='Date', usecols=['Date', 'Adj Close'], header=0, parse_dates=True)
    df = df.rename(columns={'Adj Close': os.path.split(ticker_id)[1]})

    if prices_df is not None:
        prices_df = prices_df.join(df)
    else:
        prices_df = df

prices_df = prices_df.sort_values(by='Date', axis=0)
prices_df

,AADR,AAXJ,ACCU,ACWI,ACWX,AGG,ALD,AMLP,AND,ARGT,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,YPRO,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14411.099609
2015-07-13,38.026569,55.794907,NaN,53.946297,39.022835,95.111549,41.725567,52.897274,7.804343,18.792439,...,14.085241,49.342838,38.891907,43.365269,36.300888,69.638321,73.368744,164.961609,NaN,14533.200195
2015-07-14,38.026569,55.686138,NaN,54.277966,39.242306,95.269760,41.901981,53.264389,7.890951,19.051447,...,14.119128,49.477974,39.039825,43.435162,36.250233,70.335449,73.461868,169.921738,NaN,14599.400391
2015-07-15,38.026569,55.042519,NaN,54.134541,39.066727,95.524658,41.632706,52.530167,7.775474,19.013077,...,14.237730,49.270763,39.049072,43.321602,36.427559,70.427200,73.387375,163.383377,NaN,14662.299805
2015-07-16,38.696598,55.776779,NaN,54.528973,39.365215,95.603722,41.755276,52.062946,7.736981,19.435162,...,14.378919,49.495991,39.548279,43.767128,36.967972,70.757393,73.834351,161.466980,NaN,14731.099609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-06,54.660000,75.269997,NaN,76.400002,45.139999,118.180000,NaN,23.510000,NaN,25.809999,...,23.410000,69.970001,107.110001,59.279999,57.189999,102.629997,132.580002,52.070000,NaN,15669.700195
2020-07-07,54.880001,73.839996,NaN,75.510002,44.430000,118.360001,NaN,23.660000,NaN,25.500000,...,22.930000,68.629997,105.989998,59.820000,56.959999,101.739998,130.500000,50.259998,NaN,15595.500000
2020-07-08,55.520000,75.760002,NaN,76.290001,45.029999,118.320000,NaN,23.700001,NaN,25.660000,...,23.170000,68.629997,107.720001,59.669998,57.470001,101.750000,131.889999,50.310001,NaN,15629.200195


Calculate price returns

In [8]:
returns_df = prices_df.pct_change()
returns_df

,AADR,AAXJ,ACCU,ACWI,ACWX,AGG,ALD,AMLP,AND,ARGT,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,YPRO,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008473
2015-07-14,0.000000,-0.001949,NaN,0.006148,0.005624,0.001663,0.004228,0.006940,0.011097,0.013783,...,0.002406,0.002739,0.003803,0.001612,-0.001395,0.010011,0.001269,0.030068,NaN,0.004555
2015-07-15,0.000000,-0.011558,NaN,-0.002642,-0.004474,0.002676,-0.006426,-0.013784,-0.014634,-0.002014,...,0.008400,-0.004188,0.000237,-0.002614,0.004892,0.001304,-0.001014,-0.038479,NaN,0.004308
2015-07-16,0.017620,0.013340,NaN,0.007286,0.007640,0.000828,0.002944,-0.008894,-0.004951,0.022200,...,0.009917,0.004571,0.012784,0.010284,0.014835,0.004688,0.006091,-0.011729,NaN,0.004692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.010538,0.049644,NaN,0.019210,0.023583,-0.000085,0.000000,-0.028111,0.000000,0.069180,...,0.019155,0.014352,0.017479,0.004916,-0.012263,0.010138,0.023705,0.002117,NaN,0.004674
2020-07-07,0.004025,-0.018998,NaN,-0.011649,-0.015729,0.001523,0.000000,0.006380,0.000000,-0.012011,...,-0.020504,-0.019151,-0.010457,0.009109,-0.004022,-0.008672,-0.015689,-0.034761,NaN,-0.004735
2020-07-08,0.011662,0.026002,NaN,0.010330,0.013504,-0.000338,0.000000,0.001691,0.000000,0.006275,...,0.010467,0.000000,0.016322,-0.002508,0.008954,0.000098,0.010651,0.000995,NaN,0.002161


Calucate expected return using geomean from price return

In [20]:
from scipy.stats import gmean

exp_return_df = pd.DataFrame()
for found_ticker in found_tickers:
    returns_sr = returns_df[pd.notnull(returns_df[found_ticker])][found_ticker]
    if exp_return_df.empty:
        exp_return_df = pd.DataFrame(data={
            'ticker': found_ticker,
            'exp_return': [0] if returns_sr.empty else [gmean(returns_sr + 1) - 1]
        })
    else:
        exp_return_df = pd.concat([
            exp_return_df,
            pd.DataFrame(data={
                'ticker': found_ticker,
                'exp_return': [0] if returns_sr.empty else [gmean(returns_sr + 1) - 1]
            })
        ])

exp_return_df = exp_return_df.set_index('ticker').T
exp_return_df

ticker,GTAA,ARKW,SCHC,HECO,DLN,DBB,BWX,XBI,VWO,USO,...,IWM,GVI,TLT,DJP,EMLP,FBT,PSQ,EWJ,IDV,VTI
exp_return,0.000008,0.001252,0.000079,0.00015,0.000296,0.000027,0.000122,0.000231,0.000164,-0.00123,...,0.000146,0.00013,0.000378,-0.000343,-0.00004,0.000254,-0.000861,0.000131,0.000034,0.000382


Calculate expected covariance using price return

In [12]:
covar_df = returns_df.cov()
covar_df

,AADR,AAXJ,ACCU,ACWI,ACWX,AGG,ALD,AMLP,AND,ARGT,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,YPRO,^GSPTSE
AADR,0.000191,0.000134,NaN,0.000123,0.000128,8.304965e-06,1.300385e-06,0.000155,0.000020,0.000168,...,0.000138,0.000129,0.000144,0.000073,0.000071,0.000101,0.000122,0.000190,NaN,0.000107
AAXJ,0.000134,0.000175,NaN,0.000133,0.000143,9.727179e-07,7.601095e-06,0.000142,0.000035,0.000160,...,0.000146,0.000136,0.000152,0.000077,0.000075,0.000106,0.000128,0.000205,NaN,0.000101
ACCU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACWI,0.000123,0.000133,NaN,0.000134,0.000133,5.634271e-07,5.217569e-06,0.000151,0.000024,0.000151,...,0.000161,0.000146,0.000155,0.000091,0.000092,0.000117,0.000134,0.000213,NaN,0.000106
ACWX,0.000128,0.000143,NaN,0.000133,0.000141,2.233496e-06,8.014894e-06,0.000157,0.000027,0.000156,...,0.000155,0.000140,0.000145,0.000083,0.000084,0.000106,0.000127,0.000215,NaN,0.000107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLV,0.000101,0.000106,NaN,0.000117,0.000106,-1.421955e-06,2.268029e-07,0.000116,0.000019,0.000119,...,0.000138,0.000127,0.000141,0.000089,0.000089,0.000142,0.000117,0.000160,NaN,0.000087
XLY,0.000122,0.000128,NaN,0.000134,0.000127,1.444136e-06,1.877616e-06,0.000151,0.000020,0.000150,...,0.000163,0.000150,0.000163,0.000091,0.000088,0.000117,0.000157,0.000207,NaN,0.000104
XOP,0.000190,0.000205,NaN,0.000213,0.000215,-2.394988e-06,-1.492755e-05,0.000468,0.000033,0.000296,...,0.000280,0.000252,0.000223,0.000111,0.000095,0.000160,0.000207,0.000807,NaN,0.000205
YPRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Reorder expected return to line up with covar matrix

In [22]:
exp_return_df = exp_return_df[covar_df.columns]
exp_return_df

ticker,AADR,AAXJ,ACCU,ACWI,ACWX,AGG,ALD,AMLP,AND,ARGT,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,YPRO,^GSPTSE
exp_return,0.000295,0.000232,0.0,0.000272,0.00011,0.000172,-0.005419,-0.000646,-0.001779,0.000247,...,0.000399,0.000251,0.000799,0.000258,0.000363,0.000288,0.000467,-0.000942,0.0,0.000068


Convex optimization of portfolio

In [39]:
import cvxpy as cp
import numpy as np

weights = cp.Variable(exp_return_df.columns.__len__(), name='weights')
gamma = cp.Parameter(nonneg=True)
port_return = exp_return_df.iloc[0].T * weights 
risk = cp.quad_form(weights, covar_df.to_numpy())
opt_problem = cp.Problem(cp.Maximize(port_return - gamma * risk), 
               [cp.sum(weights) == 1, 
                weights >= 0])
opt_problem.solve()
weights.value 

C:\tools\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


ValueError: setting an array element with a sequence.